# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675)\ underperformed\  compared\  to\  schools \ with \ smaller\  budgets (\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
# define some formating functions

def format_currency(df, cols):
    for col in cols: 
        df[col] = df[col].map("${:,.2f}".format)
    return df

def format_2decimal(df, cols):
    for col in cols: 
        df[col] = df[col].map("{:.2f}".format)
    return df

def format_percent(df, cols):
    for col in cols: 
        df[col] = df[col].map("{:.2f}%".format)
    return df

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
# calculate some stats for the district
num_school = len(school_data_complete['School ID'].drop_duplicates())
num_student = len(school_data_complete['Student ID'].drop_duplicates()) # count on "Student ID" since 
                                                                        # there are quite a lot of students 
                                                                        # with identical names
total_budget = school_data_complete[['School ID', 'budget']].drop_duplicates()['budget'].sum()
average_math = school_data_complete['math_score'].sum() / num_student
average_reading = school_data_complete['reading_score'].sum() / num_student

passing_math = school_data_complete[school_data_complete['math_score'] >= 70].count()["Student ID"] / \
                num_student * 100
passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].count()["Student ID"] / \
                   num_student * 100
overall_passing = (average_math + average_reading) / 2

# create a DataFrame "district_summary_df" with summary of data
district_summary_df = pd.DataFrame({"Total Schools": [num_school], 
                                   "Total Students": ["{:,d}".format(num_student)],
                                   "Total Budget": ["${:,.2f}".format(total_budget)],
                                   "Average Math Score" : ["{:.2f}".format(average_math)],
                                   "Average Reading Score" : ["{:.2f}".format(average_reading)],
                                   "% Passing Math" : ["{:.2f} %".format(passing_math)],
                                   "% Passing Reading" : ["{:.2f} %".format(passing_reading)],
                                   "% Overall Passing Rate" : ["{:.2f} %".format(overall_passing)]})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98 %,85.81 %,80.43 %


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
# creat a group object by "school_name"
school_grouped = school_data_complete.groupby("school_name")

# creat an empty summary df, and fill in corresponding contents
school_summary_df = pd.DataFrame()

school_summary_df["School Type"]  = school_grouped.first()["type"]

school_summary_df["Total Students"] =  school_grouped.count()["Student ID"]

school_summary_df["Total School Budget"] = school_grouped.first()["budget"]

school_summary_df["Per Student Budget"] = (school_summary_df["Total School Budget"] / \
                                           school_summary_df["Total Students"])

school_summary_df["Average Math Score"] = school_grouped.mean()["math_score"]

school_summary_df["Average Reading Score"] = school_grouped.mean()["reading_score"]

# first filter the DataFrame "school_data_complete" with conditions (obtain students with passing score), 
# then group by "school_name"
# and calcualte passing rate

con = school_data_complete["math_score"] >= 70
school_summary_df["% Passing Math"] = (school_data_complete[con].groupby("school_name").count()["Student ID"]/ \
                                       school_summary_df["Total Students"]) * 100

con = school_data_complete["reading_score"] >= 70
school_summary_df["% Passing Reading"] = (school_data_complete[con].groupby("school_name").count()["Student ID"]/ \
                                       school_summary_df["Total Students"]) * 100

# avarage passing rate
school_summary_df["Overall Passing Rate"] = (school_summary_df["% Passing Math"] + 
                                             school_summary_df["% Passing Reading"]) / 2

del school_summary_df.index.name

# keep a copy of the nonformated version
school_summary_df_num = school_summary_df.copy()

# format the DataFrame 
cols = ["Total School Budget", "Per Student Budget"]
school_summary_df = format_currency(school_summary_df, cols)

cols = ["Average Math Score", "Average Reading Score"]
school_summary_df = format_2decimal(school_summary_df, cols)

cols = ["% Passing Math", "% Passing Reading", "Overall Passing Rate"]
school_summary_df = format_percent(school_summary_df, cols)

school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
# sort DataFrame "school_summary_df" by "Overall Passing Rate"
top_school_df = school_summary_df_num.sort_values(by = "Overall Passing Rate", ascending = False)

# format the dataframe
cols = ["Total School Budget", "Per Student Budget"]
top_school_df = format_currency(top_school_df, cols)

cols = ["Average Math Score", "Average Reading Score"]
top_school_df = format_2decimal(top_school_df, cols)

cols = ["% Passing Math", "% Passing Reading", "Overall Passing Rate"]
top_school_df = format_percent(top_school_df, cols)

top_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
# sort DataFrame "school_summary_df" by "Overall Passing Rate"
worst_school_df = school_summary_df_num.sort_values(by = "Overall Passing Rate", ascending = True)

# format the dataframe
cols = ["Total School Budget", "Per Student Budget"]
worst_school_df = format_currency(worst_school_df, cols)

cols = ["Average Math Score", "Average Reading Score"]
worst_school_df = format_2decimal(worst_school_df, cols)

cols = ["% Passing Math", "% Passing Reading", "Overall Passing Rate"]
worst_school_df = format_percent(worst_school_df, cols)

worst_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# obtain all grades from the DataFrame "school_data_complete"
grades = school_data_complete["grade"].value_counts()
grades_list = list(grades.index)

# print all avaialble grades
print("Grades: " + " ".join(grade for grade in grades_list))

Grades: 9th 10th 11th 12th


In [9]:
# create new empty DataFrame
math_score_grade_df = pd.DataFrame()

# for each grade, create a condition for filtering 
# then groupby "school_name" and calculate the average "math_score"
# append the new column to DataFrame "math_score_grade_df"
for grade in grades_list: 
    con = school_data_complete["grade"] == grade
    math_score_grade_df[grade] = school_data_complete[con].groupby("school_name").mean()["math_score"]

del math_score_grade_df.index.name

# formatting, 2 demical is enough 
cols = math_score_grade_df.columns
math_score_grade_df = format_2decimal(math_score_grade_df, cols)

math_score_grade_df

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
# create new empty DataFrame
reading_score_grade_df = pd.DataFrame()

# for each grade, create a condition for filtering 
# then groupby "school_name" and calculate the average "reading_score"
# append the new column to DataFrame "reading_score_grade_df"
for grade in grades_list: 
    con = school_data_complete["grade"] == grade
    reading_score_grade_df[grade] = school_data_complete[con].groupby("school_name").mean()["reading_score"]

del reading_score_grade_df.index.name

# formatting, 2 demical is enough 
cols = reading_score_grade_df.columns
reading_score_grade_df = format_2decimal(reading_score_grade_df, cols)

reading_score_grade_df

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
# create bins for school spending on "per student budget"
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# add a column to the DataFrame "school_summary_df_num" based on the school's spending on each student
school_summary_df_num["Spending Group"] = pd.cut(school_summary_df_num["Per Student Budget"], 
                                                 spending_bins, 
                                                 labels = group_names)
school_summary_df_num.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Group
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645


In [12]:
# group the DataFrame by "Spending Group"
# select the DataFrame with the desire columns
cols = ['Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 'Overall Passing Rate']
school_spending_df = school_summary_df_num.groupby("Spending Group").mean()[cols]
school_spending_df.index.name = "Spending Ranges (Per Student)" # change index name
school_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
# Select bins based on school size
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# add a column to the DataFrame "school_summary_df_num" based on the school's spending on each student
school_summary_df_num["School Size Group"] = pd.cut(school_summary_df_num["Total Students"], 
                                                 size_bins, 
                                                 labels = group_names)
school_summary_df_num.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Group,School Size Group
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,$615-645,Large (2000-5000)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,<$585,Medium (1000-2000)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,$615-645,Large (2000-5000)
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,$615-645,Large (2000-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645,Medium (1000-2000)


In [14]:
school_size_df = school_summary_df_num.groupby("School Size Group").mean()[cols]
school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size Group,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [15]:
school_type_df = school_summary_df_num.groupby("School Type").mean()[cols]
school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
